In [48]:
import pandas as pd

# datset:  https://www.kaggle.com/datasets/meetnagadia/amazon-kindle-book-review-for-sentiment-analysis/code?datasetId=1569305&sortBy=dateRun&tab=profile&excludeNonAccessedDatasources=false

df= pd.read_csv("all_kindle_review .csv")

df.head()


KeyboardInterrupt



In [ ]:
df.columns

In [ ]:
df= df[["reviewText", "rating"]]

df.head()

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
df["rating"].value_counts()

In [ ]:
#consider all review less than 3 are unhappy(0), others are happy
(1)
df["rating"] = df["rating"].apply(lambda x: 0 if x<3 else 1)

In [ ]:
df["rating"].value_counts()

In [ ]:
df["reviewText"]=df["reviewText"].str.lower()

In [ ]:
df.head()

In [ ]:
import re

import nltk

from nltk.corpus import stopwords

nltk.download("stopwords")

In [53]:
!pip install bs4

  Using cached bs4-0.0.2-py2.py3-none-any.whl.metadata (411 bytes)
Using cached bs4-0.0.2-py2.py3-none-any.whl (1.2 kB)


In [56]:
from bs4 import BeautifulSoup

In [59]:
df["reviewText"]=df["reviewText"].apply(lambda x: re.sub('[^a-z A-Z 0-9-]+',"", x))

df["reviewText"]=df["reviewText"].apply(lambda x: " ".join([y for y in x.split() if y not in stopwords.words("english")]))


df["reviewText"]=df["reviewText"].apply(lambda x: re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', "", str(x)))

df["reviewText"]=df["reviewText"].apply(lambda x: BeautifulSoup(x,"lxml").get_text())

df["reviewText"]=df["reviewText"].apply(lambda x: " ".join(x.split()))

In [60]:
df.head()

,reviewText,rating
0,jace rankin may short hes nothing mess man hau...,0
1,great short read didnt want put read one sitti...,0
2,ill start saying first four books wasnt expect...,0
3,aggie angela lansbury carries pocketbooks inst...,0
4,expect type book library pleased find price right,0


In [61]:
from nltk.stem import WordNetLemmatizer

In [62]:
lemmatize=WordNetLemmatizer()

In [63]:
def lemm_word(text):
    return " ".join([lemmatize.lemmatize(word) for word in text.split()])

In [65]:
df["reviewText"]=df["reviewText"].apply(lambda x:lemm_word(x) )

In [66]:
df.head()

,reviewText,rating
0,jace rankin may short he nothing mess man haul...,0
1,great short read didnt want put read one sitti...,0
2,ill start saying first four book wasnt expecti...,0
3,aggie angela lansbury carry pocketbook instead...,0
4,expect type book library pleased find price right,0


In [69]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df["reviewText"], df["rating"], test_size=0.2)

In [70]:
!pip install gensim

In [80]:
from sklearn.feature_extraction.text import CountVectorizer

bow = CountVectorizer()

X_train_bow= bow.fit_transform(X_train).toarray()

X_test_bow= bow.transform(X_test).toarray()

In [81]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()

X_train_tfidf= tfidf.fit_transform(X_train).toarray()

X_test_tfidf= tfidf.transform(X_test).toarray()

In [82]:
print(X_train_bow.shape,X_test_bow.shape )
print(X_train_tfidf.shape,X_test_tfidf.shape )

(9600, 36046) (2400, 36046)
(9600, 36046) (2400, 36046)


In [83]:
# applying model on it

from sklearn.naive_bayes import GaussianNB

model = GaussianNB()

bow_model= model.fit(X_train_bow, y_train)
tfidf_model=model.fit(X_train_tfidf, y_train)

In [84]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [85]:
bow_pred = bow_model.predict(X_test_bow)
tfidf_pred = tfidf_model.predict(X_test_tfidf)

In [87]:
bow={}

bow["confusion"] = confusion_matrix(y_test, bow_pred)

bow["accuracy"] = accuracy_score(y_test, bow_pred)

bow["classification"] = classification_report(y_test, bow_pred)

print(bow)

{'confusion': array([[2400]]), 'accuracy': 1.0, 'classification': '              precision    recall  f1-score   support\n\n           0       1.00      1.00      1.00      2400\n\n    accuracy                           1.00      2400\n   macro avg       1.00      1.00      1.00      2400\nweighted avg       1.00      1.00      1.00      2400\n'}


/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:395: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(


In [88]:
tfidf={}

tfidf["confusion"] = confusion_matrix(y_test, tfidf_pred)

tfidf["accuracy"] = accuracy_score(y_test, tfidf_pred)

tfidf["classification"] = classification_report(y_test, tfidf_pred)

print(tfidf)

{'confusion': array([[2400]]), 'accuracy': 1.0, 'classification': '              precision    recall  f1-score   support\n\n           0       1.00      1.00      1.00      2400\n\n    accuracy                           1.00      2400\n   macro avg       1.00      1.00      1.00      2400\nweighted avg       1.00      1.00      1.00      2400\n'}


/opt/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:395: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(


In [ ]:
# word2vec implementation for the same

In [89]:

import nltk
from nltk.tokenize import word_tokenize

sentences = df['reviewText'].tolist()

nltk.download('punkt')

tokenized_sentences = [word_tokenize(sentence) for sentence in sentences]



[nltk_data] Downloading package punkt to /Users/apple/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [90]:
from gensim.models import Word2Vec

# Initialize and train the Word2Vec model
model = Word2Vec(sentences=tokenized_sentences, vector_size=100, window=5, min_count=1, workers=4)

# Save the model for later use
model.save("word2vec.model")

In [91]:
import numpy as np

def get_sentence_vector(model, sentence):
    # Filter words in the model's vocabulary
    words = [word for word in sentence if word in model.wv]
    if len(words) == 0:
        return np.zeros(model.vector_size)
    return np.mean([model.wv[word] for word in words], axis=0)

sentence_vectors = [get_sentence_vector(model, sentence) for sentence in tokenized_sentences]

In [93]:
X_train, X_test, y_train, y_test = train_test_split(sentence_vectors, df['rating'].tolist(), test_size=0.2, random_state=42)




In [94]:
gnb = GaussianNB()
gnb.fit(X_train, y_train)

# Make predictions
y_pred = gnb.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)

In [95]:
print(accuracy)

1.0
